# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 30, 2, 52, 32, 372570, tzinfo=TzInfo(-05:00)),
   'digest': '77e7d26c05df9acac668c43ac2f3f6f2545110ae0ad88ec0bc48c93ea5e6675a',
   'size': 19762152260,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen3',
    'families': ['qwen3'],
    'parameter_size': '32.8B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=Tz

In [2]:
import ollama

model = "hf.co/unsloth/Qwen3-32B-GGUF:latest"

# ollama.pull(model) # uncomment this if you need to download the model

# generating a new story

In [3]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

completions = []
story_variables = []

In [4]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)

completions.append(response.model_dump())
premise_opening = response.message.content

Markdown(response.message.content)

<think>
Okay, the user wants me to create a story premise and an opening paragraph based on their guidelines. Let me start by understanding the requirements. They mentioned that if there are no user ideas, I should come up with a fresh, unexpected combination of genre, setting, protagonist, conflict, and twist. Since the user hasn't provided any specific ideas, I need to invent something original.

First, let me brainstorm some elements. The genre could be a mix of sci-fi and mystery. The setting might be a space station to give it a unique backdrop. For the protagonist, maybe someone with a unique ability, like a memory hacker. The conflict could involve a murder mystery where the protagonist has to navigate both the technical aspects and personal stakes. The twist could be that the victim's memory reveals a conspiracy or something personal.

Now, the premise needs to be concise, with stakes and a hook. Let me try: "When a memory hacker on a decaying space station is forced to solve a murder she didn't witness, she uncovers a conspiracy that erases the line between victim and perpetrator—while her own stolen past threatens to unravel her." That includes the protagonist's job, the setting, the conflict, and a twist involving her own past.

For the opening paragraph, I need a vivid scene. Let's start with her working on a memory, maybe in a gritty environment. Show her skills, introduce the murder through a client, and end with a hook about her own memories. Need to keep it around 150 words, use strong imagery, and avoid clichés. Maybe something like her extracting a memory, then getting a call about a murder. The victim's memory is corrupted, and she finds a personal connection. End with her realizing the victim had a piece of her own past.

Check for tone: should be suspenseful, maybe a bit dark but not too explicit. POV is first person to make it personal. Make sure the opening sets up the mystery and the personal stake. Avoid copying phrases, use original descriptions. Let me put that all together now.
</think>

Premise: When a memory-hacker private investigator on a decaying space station is hired to solve a murder she never witnessed, she uncovers a conspiracy that blurs the line between victim and perpetrator—while her own stolen past begins to leak into the case like a virus.  

Opening paragraph: The memory stick twitched in my hand like a live wire. I’d just extracted a security guard’s recollection of the crime scene—a routine job, until the station’s AI flagged the victim’s neural signature as *mine*. The station’s flickering holograms cast blue shadows over the corpse in the morgue, their light glinting off the frosty haze of his frozen eyes. Client wanted answers, credits upfront, and a report by morning. But when I probed the victim’s final moments, his memory dissolved into static—a rare, deliberate block. I’d seen that pattern before: the same glitch that haunted my own fragmented past. My fingers froze on the keyboard. Somewhere in this labyrinth of rust and code, someone was erasing truths—and now a dead man’s last breath smelled like my shampoo.

In [5]:
import re

text = premise_opening  # your input string

pattern = re.compile(r"""
^\s*
(?:[\*_]{1,3}\s*)?                                 # optional opening markdown (** or *)
(?P<label>premise|opening\s+paragraph|original\s+paragraph)
[\s\*_]*[:：\-—–]?[\s\*_]*                          # optional punctuation/spacing around label
(?P<body>.+?)                                      # capture the body non-greedily
(?=
    ^\s*(?:[\*_]{1,3}\s*)?
        (?:premise|opening\s+paragraph|original\s+paragraph)[\s\*_]*[:：\-—–]?  # next labeled section
  | \n{2,}(?=\S)                                   # or a blank-line block then non-space
  | \Z                                             # or end of string
)
""", flags=re.IGNORECASE | re.MULTILINE | re.DOTALL | re.VERBOSE)

label_norm = {
    "premise": "Premise",
    "opening paragraph": "Opening paragraph",
    "original paragraph": "Original paragraph",
}

sections = {}
for m in pattern.finditer(text):
    raw_label = re.sub(r"\s+", " ", m.group("label").lower()).strip()
    label = label_norm.get(raw_label, raw_label.title())
    body = m.group("body").strip()
    sections[label] = f"{label}: {body}"

premise   = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

story_variables.append({"premise": premise})
story_variables.append({"paragraph": paragraph})

print(premise)
print(paragraph)


Premise: When a memory-hacker private investigator on a decaying space station is hired to solve a murder she never witnessed, she uncovers a conspiracy that blurs the line between victim and perpetrator—while her own stolen past begins to leak into the case like a virus.
Opening paragraph: The memory stick twitched in my hand like a live wire. I’d just extracted a security guard’s recollection of the crime scene—a routine job, until the station’s AI flagged the victim’s neural signature as *mine*. The station’s flickering holograms cast blue shadows over the corpse in the morgue, their light glinting off the frosty haze of his frozen eyes. Client wanted answers, credits upfront, and a report by morning. But when I probed the victim’s final moments, his memory dissolved into static—a rare, deliberate block. I’d seen that pattern before: the same glitch that haunted my own fragmented past. My fingers froze on the keyboard. Somewhere in this labyrinth of rust and code, someone was erasin

In [6]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [7]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)

completions.append(response.model_dump())
continued_paragraph = response.message.content

story_variables.append({"continued_paragraph":continued_paragraph})

Markdown(response.message.content)

<think>
Okay, let's tackle this. The user wants me to continue the story based on the given premise and opening paragraph. First, I need to parse the key elements from the premise and the opening. The main character is a memory-hacker PI on a decaying space station. The murder case she's investigating has a connection to her own stolen past. The tone is noir, with elements of sci-fi and mystery. The POV is close third person, past tense.

The opening paragraph ends with the protagonist finding a connection between the victim and her own past, specifically mentioning her shampoo. The next scene needs to move the plot forward, so I need to think about what comes next. The user provided a sample response, so I need to make sure my thinking aligns with that structure.

First, extract continuity: the protagonist's name isn't given, but maybe she has a call sign or a name mentioned elsewhere? The victim's neural signature is hers, which suggests she might have a connection to him. The memory block is deliberate, pointing to a conspiracy. The stakes are high because her own past is involved, and the client wants answers quickly. The setting is a decaying space station, so the environment can be used to show the decay and maybe technical issues that affect the investigation.

For the mini beat plan, the next scene should escalate the mystery. Maybe she investigates the victim's background, finds a link to her past, encounters resistance from the station's authorities or the client, discovers more about the conspiracy, and uncovers a personal connection. The sample response had her checking the victim's identity, finding he was a former colleague, facing interference, accessing hidden files, and finding a personal message. The cliffhanger was the security breach and the unknown pursuer.

I need to outline similar beats. Let me think: 1. Protagonist checks victim's identity, finds a link to her past. 2. Discovers he was part of a project related to her stolen memories. 3. Tries to access his files but faces technical obstacles. 4. Finds a hidden message or clue pointing to a larger conspiracy. 5. Encounters someone who knows about her past, leading to conflict. 6. Realizes she's being watched or hunted. 7. Cliffhanger with a threat or revelation.

Now, writing the scene. Start with her investigating the victim's background. Maybe she uses her hacking skills to dig deeper. Since the AI flagged the neural signature, perhaps she accesses the station's database but finds inconsistencies. The victim's records are altered or missing. She might find a name that triggers her fragmented memories. Then, she could go to a specific location connected to the victim, where she encounters someone hostile or finds more clues. The scene should show her determination but also the danger she's in. Use dialogue to reveal motives, maybe a contact or a suspect. The tone should stay tense, with descriptions of the decaying station to emphasize the setting. End with a soft cliffhanger, like an approaching threat or a new mystery element.

Make sure to use close third person, past tense. Show her emotions through actions and thoughts. Maybe include technical jargon related to memory-hacking and space stations to keep the sci-fi elements authentic. Avoid clichés, focus on concrete details. The sample used elements like the station's decay, the AI's interference, and personal connections. I should follow that, ensuring continuity and escalating the plot.
</think>

The victim’s ID file dissolved like smoke in her neural interface. Again. She spat the taste of rusted coolant and frustration as she reran the query, her fingers drumming the frostbitten console. The station’s archives were a patchwork of corrupted sectors—*of course* the dead man’s records were buried in the most decayed quadrant. A flicker of movement in the peripheral feed: a janitor’s droid scrubbing biometric locks down the hall. Not a droid. A *person* in droid cover, their silhouette too sharp against the grime. She yanked her visor down, rerouted her queries through the ventilation system’s forgotten maintenance ports.  

The name crystallized at last: **Elias Vey**, ex-neurotech engineer. A pulse of recognition—*his hands, cold on her temples, a voice saying, “This will hurt less if you’re asleep”*—flickered in her gut. His file was a void, but his job at the station’s memory banks wasn’t. She traced his last login to Sector 7’s abandoned cryo-wing, where the air reeked of ozone and decay. Her boots crunched over shattered data chips as she pried open the wing’s access panel. Inside, Elias’s workstation glowed faintly, its interface humming a dissonant tune.  

A voice message blinked to life. *“If you’re hearing this, you’ve already forgotten me.”* The speaker’s accent was clipped, academic—*hers*, if she could trust the echo of it in her own mind. “You were part of a project,” Elias continued, his tone fraying. “They called it *Reclamation*. They’ll come for you next. Don’t trust the client.” The feed died. Her breath fogged the screen. *Reclamation*. The word burrowed under her ribs, a splinter of a memory: white rooms, needles, a child’s scream that wasn’t hers.  

A clang echoed behind her. She spun, weapon drawn—a janitor’s droid, its scrubbing arm tangled in a tangle of cables. But the droid’s camera lens was *winking*, a deliberate blink. She smashed it with the butt of her gun. Static hissed, then a voice: *“They know you’re here.”* Not Elias. Female. Smaller. A name tugged at her—*Lira? Lira something*—a face flickering at the edge of her mind, dissolved before she could grasp it.  

The cryo-wing’s emergency lights flared red. She sprinted for the exit, but the doors locked mid-stride. A new alert flashed on her visor: *Security breach detected. Containment protocols engaged.* Her comm buzzed—a priority call from the client. “Investigator,” the client’s voice purred, genderless and smooth as black ice. “You’ve been… *thorough*. Cease your inquiry. Now.” She severed the link. Too late. The air trembled as the station’s automated sentries materialized in the corridor, their optics glowing like predators.  

She vaulted over a console, slamming her palm against Elias’s terminal. A final command executed: the cryo-wing’s data core began uploading to the black market relay she’d always kept buried. Somewhere, a buyer would get her files. *Good.* Let them parse the mess of her stolen years. She hit the emergency泄压 valve—a desperate, loud hiss as air rushed out. The sentries staggered, recalibrating. She slipped into the shadows, her pulse roaring.  

The voice in the droid’s static had been a message, not a warning. *Lira.* The name clung now, a thread. She’d find the woman who’d once saved—or damned—her. But first, she had to survive the station’s next *Reclamation*.  

---  
**Beat plan**:  
1. Protagonist uncovers victim’s identity and hidden project link.  
2. Memory fragment reveals shared history with Elias.  
3. Security breach forces her to flee, escalating danger.  
4. Mysterious ally’s message hints at deeper conspiracy.  
5. Protagonist sacrifices data to escape, setting up next pursuit.  
**Cliffhanger**: The client’s direct threat and the “Lira” lead force her into a deadlier game.

In [8]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [9]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)

completions.append(response.model_dump())
summary = response.message.content

story_variables.append({"summary":summary})

Markdown(response.message.content)

<think>
Alright, let's tackle this. The user provided a detailed story and wants a "Story-So-Far" handoff. First, I need to extract all the key elements from the given text and the initial analysis. The main character is a memory-hacking PI, right? Her name isn't given, but maybe she has a call sign? The setting is a decaying space station with AI and holograms. The conflict involves a murder that's tied to her own past, specifically her fragmented memories and a project called "Reclamation."

I need to outline the cast, relationships, world rules, POV, timeline, current situation, active threads, and so on. Let me start with the logline. It should be a concise summary. The cast includes the protagonist, Elias Vey, the client, and maybe Lira. The world rules involve memory hacking, neural signatures, and the decaying station's tech. The POV is close third, past tense, with a noir tone. The timeline includes the discovery of the victim's neural signature, the memory block, and the chase scene. Current situation is her evading security after uncovering the Reclamation project.

Active threads would be the Reclamation project, the client's true intentions, Lira's role, and the protagonist's past. Continuity landmines include the name "Lira" and the project name. Ambiguities might be the protagonist's full name and the exact nature of Reclamation. Style DNA includes noir elements and tech jargon. Next-scene seeds could involve pursuing Lira, the client's next move, or the data core's contents.

Wait, the user's sample response had specific sections like Logline, Cast & relationships, etc. I need to make sure each section is filled out accurately without adding new info. Let me check the original text again to ensure I didn't miss any key points. The victim's memory had her shampoo scent, which is a personal connection. The client is genderless and smooth, which is a detail to include. The droid's message points to Lira. The Reclamation project is central. Also, the station's security protocols and the protagonist's hacking skills are part of the world rules.

I need to avoid inventing anything. The current situation is her escaping the cryo-wing with the data uploaded. Active threads include the client's threat, Reclamation, Lira, and the protagonist's past. Continuity landmines are the name spellings and the project name. Ambiguities: protagonist's name and Reclamation's full scope. Style elements include noir tone and tech terms.

Next-scene seeds should be beats for the next part. Maybe she tracks Lira, the client sends more threats, or the data core's info reveals more. Need to keep it non-prose and concise. Okay, I think that covers all the required sections. Let me structure it properly now.
</think>

**Logline**  
A memory-hacking PI on a decaying space station uncovers a murder tied to her own stolen past, triggering a lethal game of cat-and-mouse with a faceless client and a conspiracy called *Reclamation*.  

**Cast & relationships**  
* **Protagonist (unnamed)**: Memory-hacker PI; seeks to uncover her erased past while evading a client and station security. Ties to Elias Vey (victim), Lira (mysterious ally), and the client (antagonist).  
* **Elias Vey**: Ex-neurotech engineer; victim of murder; his files hint at shared history with the protagonist.  
* **Client**: Genderless, smooth-voiced employer; orchestrates the *Reclamation* project; directly threatens the protagonist.  
* **Lira**: Mysterious figure from protagonist’s past; mentioned in a cryptic droid-message; potential ally or saboteur.  

**World/Rules**  
* Memory-hacking involves extracting neural signatures; memories can be deliberately blocked or erased.  
* The station’s AI and security systems are unreliable, with decaying tech and patchwork archives.  
* *Reclamation* is a classified project tied to memory manipulation; its goals remain unclear.  
* Conflicts are resolved through tech sabotage, stealth, and psychological warfare (no guns).  

**POV/Tense/Tone**  
* POV: Close third on the protagonist  
* Tense: Past  
* Tone/Style: Noir-infused sci-fi; tense, atmospheric; focuses on sensory decay (rust, ozone, static).  

**Timeline & Causality**  
1. Protagonist discovers victim’s neural signature matches hers → triggers investigation.  
2. Victim’s memory block mirrors her own fragmented past → personal stakes escalate.  
3. Protagonist traces victim to Sector 7’s cryo-wing → uncovers *Reclamation* project.  
4. Elias’s voice message warns her not to trust the client → protagonist realizes she’s a target.  
5. Security breach forces her to flee → data core uploaded, client intervenes directly.  

**Current Situation**  
The protagonist has fled the cryo-wing after uploading stolen data, but the station’s automated sentries are hunting her. She’s pursued by the client’s forces and must locate Lira, the woman from her fragmented memories, to unravel *Reclamation*. Immediate stakes: survive the station’s security protocols and decode Elias’s cryptic warning.  

**Active Threads / Hooks**  
* What is *Reclamation* and why does the protagonist’s past intersect with it?  
* Who is Lira, and why does she hold the key to the protagonist’s memories?  
* The client’s true motives remain obscured; are they trying to kill her or recruit her?  
* The droid’s message (“They know you’re here”) hints at a third faction with its own agenda.  

**Continuity Landmines**  
* Protagonist’s name is intentionally withheld; ensure no contradictions in her identity.  
* *Reclamation* must remain ambiguously defined until later reveals.  
* Lira’s role and Elias’s prior relationship to the protagonist are unexplained.  
* Station’s tech rules (e.g., no guns, memory-hacking mechanics) must not be violated.  

**Ambiguities / Gaps**  
* Protagonist’s real name: Assume it’s intentionally erased by *Reclamation*; no need to invent it.  
* Lira’s allegiance: Treat as ambiguous until resolved by future clues.  

**Style DNA**  
* Noir cadence with sci-fi grit: “The air reeked of ozone and decay,” “static hissed like a predator’s breath.”  
* Sensory focus on decay and technology: rust, static, flickering lights.  

**Next-Scene Seeds**  
* Option A: Protagonist tracks Lira to a black-market memory bazaar, only to find the client already waiting.  
* Option B: The uploaded data core attracts a rival hacker’s attention, forcing a tense alliance.  
* Option C: Protagonist discovers a hidden cryo-pod in Sector 7 containing a frozen version of herself.  
* Option D: Elias’s voice message includes a timestamp matching the protagonist’s earliest memory gap.  
* Option E: Station security’s sentries are reprogrammed by *Reclamation*—they now target anyone with intact memories.

In [10]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [11]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)

completions.append(response.model_dump())
continued_story = response.message.content

story_variables.append({"continued_story":continued_story})

Markdown(response.message.content)

<think>
Okay, let me try to work through this. The user wants me to continue the story based on the given premise and the initial paragraph provided. First, I need to make sure I understand the key elements from the existing text. The protagonist is a memory-hacking PI on a decaying space station, investigating a murder that's tied to her own stolen past. The tone is noir with sci-fi elements, and the POV is close third person, past tense.

The initial paragraph ends with the protagonist realizing that the victim's neural signature matches her own and that she's being pursued. The next scene needs to move the plot forward, so I should think about what comes next. The user provided a sample response, so I should look at that for structure and style clues.

In the sample, the protagonist investigates the victim's background, finds a connection to her past, encounters a droid with a message, and faces security. The key elements are the mystery of the Reclamation project, the client's involvement, and the protagonist's fragmented memories.

For the next scene, I need to escalate the tension. Maybe the protagonist finds a hidden message or clue that leads her to a new location on the station. She might encounter someone who knows her past, or the client's forces close in. Since the station is decaying, maybe there's a technical malfunction that complicates her escape or investigation.

I should also maintain the noir tone with descriptive language about the setting—rust, static, flickering lights. The protagonist's internal conflict about her stolen memories should be hinted at, maybe through sensory details or fragmented thoughts.

I need to end the scene with a soft cliffhanger, perhaps a new threat or a revelation that propels her into the next scene. Maybe she finds a hidden room or another victim, or the client's motives become clearer. Let me outline a few beats: 

1. Protagonist follows the lead from the droid's message to a specific location.
2. She discovers a hidden archive or another victim's memory.
3. The client's enforcers corner her, leading to a confrontation.
4. She uses her hacking skills to escape but leaves a trail for the client.
5. The cliffhanger involves a new clue or a personal connection to her past.

I need to ensure continuity with the previous text, so names like Elias and Reclamation should be mentioned. The setting details from the initial paragraph (decaying station, AI, security) should be consistent. Also, avoid introducing new elements that aren't hinted at yet.

Now, writing the scene with close third person, past tense, strong verbs, and dialogue that reveals conflict. Maybe include a tense interaction with another character or a chase sequence. End with a cliffhanger that makes the reader want to know what happens next.
</think>

The station’s skeletal corridors groaned as she sprinted, her breath fogging in the thin air. The droid’s message—*Lira*—had lodged itself in her skull, a splinter she couldn’t ignore. Sector 7’s cryo-wing was a dead end, but the relay’s data core had pinged a secondary cache in the **Bio-archival Wing**, a place the station’s AI had flagged as *decommissioned*. *Decommissioned?* That meant someone had gone to great lengths to erase it.  

She ducked into the maintenance shaft, her boots crunching over loose circuitry. The Bio-archival Wing’s entrance was sealed behind a biometric lock—*Elias’s signature*, she realized, the same one on the cryo-wing’s terminal. Her fingers hovered over the override, but a whisper of doubt prickled her neck. *Why would he lock it?* The answer came in a flicker of memory: *“You can’t unsee this.”* She bypassed the lock with a stolen keyframe from his terminal. The door hissed open.  

Inside, rows of stasis pods lined the walls, each one a sealed coffin. At their centers: **subjects**. Not just bodies. **People**. One pod’s visor was fogged, but the hand inside clenched and unclenched—a heartbeat. Her stomach dropped. The label read **Subject 47: L.** The rest was smudged. *Lira?* A child’s voice echoed in her skull—*“You’re not supposed to remember me!”*—but before she could pin the memory, a shadow moved.  

A figure stood at the far end of the room, silhouetted by emergency lights. Their face was obscured by a cracked visor, but their voice was a blade: **“You shouldn’t have come here.”** The station’s AI screamed a lockdown sequence. She fired, the shot glancing off a pod. The figure didn’t flinch. Instead, they yanked a control panel, and the room flooded with light.  

The visor lifted.  

**Her own face** stared back.  

The doppelgänger’s eyes were hollow, a surgical patch over one. “You’re running out of time,” the clone said, voice flat. “They’ll overwrite you next.” A pulse of static, and the pod labeled *Subject 47* shuddered, its occupant stirring. The clone lunged. The protagonist ducked, slamming the panel again. The room’s power died. In the darkness, she fled, the clone’s laughter chasing her into the maze of corridors.  

The client’s voice buzzed in her comm—*“You’re bleeding out, Investigator. Stop digging.”* She severed the link, her hands shaking. *Lira was a person. A real person.* The station’s AI, which had always whispered compliance, now hissed warnings. She was out of hiding places.  

But the Bio-archival Wing had given her something the client hadn’t anticipated: **proof**. And a clone who knew her name.  

---  
**Beat plan**:  
1. Protagonist follows the droid’s lead to a hidden archive.  
2. Discovers stasis pods containing *people*, including a clone of herself.  
3. Confrontation with the clone reveals the Reclamation project’s true nature.  
4. The client’s pursuit escalates, but the protagonist now holds irrefutable evidence.  
**Cliffhanger**: The clone’s existence and the pods’ contents hint at a deeper, personal conspiracy.

# stats

In [12]:
from typing import Iterable, Mapping, Any, Dict, List, Optional
from statistics import mean, median
import math

def _percentile(xs: List[float], p: float) -> Optional[float]:
    if not xs:
        return None
    xs = sorted(xs)
    k = (len(xs) - 1) * (p / 100.0)
    f, c = math.floor(k), math.ceil(k)
    if f == c:
        return xs[int(k)]
    return xs[f] + (xs[c] - xs[f]) * (k - f)

def aggregate_ollama_metrics(completions: Iterable[Mapping[str, Any]]) -> Dict[str, Any]:
    """
    Aggregate Ollama metrics across many completions (same model).
    Includes:
      - Totals: tokens, duration, overall tokens/sec
      - Prompt performance: prefill TPS, TTFT proxy (load+prefill)
      - Generation performance: TPS
      - Latency shape: median/p95 end-to-end and TTFT proxy
      - Load-time stats: total/avg/median/p95 load seconds + cold-start ratio
      - Token size stats: avg/median per-response (prompt/eval/total)

    Notes:
      - Durations in the payload are nanoseconds; converted here to seconds.
      - Skips items with done=False.
    """
    model = None
    counted = 0

    total_prompt_tokens = 0
    total_eval_tokens = 0
    total_duration_ns = 0

    # Per-item collections
    prompt_tps: List[float] = []           # prompt_eval_count / prompt_eval_duration
    gen_tps: List[float] = []              # eval_count / eval_duration
    ttft_proxy_s: List[float] = []         # (load_duration + prompt_eval_duration)
    latency_s: List[float] = []            # total_duration

    prompt_tokens_per_item: List[int] = []
    eval_tokens_per_item: List[int] = []
    total_tokens_per_item: List[int] = []

    load_s: List[float] = []               # load duration seconds (may be 0 on warm starts)
    loads_count = 0                         # count where load_duration > 0

    for c in completions:
        if c.get("done", True) is False:
            continue

        if model is None:
            model = c.get("model")

        p_cnt = int(c.get("prompt_eval_count", 0) or 0)
        e_cnt = int(c.get("eval_count", 0) or 0)

        p_ns  = int(c.get("prompt_eval_duration", 0) or 0)
        e_ns  = int(c.get("eval_duration", 0) or 0)
        l_ns  = int(c.get("load_duration", 0) or 0)
        t_ns  = int(c.get("total_duration", 0) or 0)

        # Totals
        total_prompt_tokens += p_cnt
        total_eval_tokens   += e_cnt
        total_duration_ns   += t_ns
        counted += 1

        # Per-item tokens
        prompt_tokens_per_item.append(p_cnt)
        eval_tokens_per_item.append(e_cnt)
        total_tokens_per_item.append(p_cnt + e_cnt)

        # Prefill & generation rates
        if p_ns > 0:
            prompt_tps.append(p_cnt / (p_ns / 1e9))
        if e_ns > 0:
            gen_tps.append(e_cnt / (e_ns / 1e9))

        # TTFT proxy & latency
        ttft_proxy_s.append((l_ns + p_ns) / 1e9)
        latency_s.append(t_ns / 1e9 if t_ns else 0.0)

        # Load time stats
        l_s = l_ns / 1e9 if l_ns else 0.0
        load_s.append(l_s)
        if l_ns > 0:
            loads_count += 1

    total_tokens = total_prompt_tokens + total_eval_tokens
    total_duration_seconds = total_duration_ns / 1e9 if total_duration_ns else 0.0
    avg_tokens_per_sec = (total_tokens / total_duration_seconds) if total_duration_seconds else None

    # Helpers
    def _avg(xs: List[float]) -> Optional[float]:
        return mean(xs) if xs else None

    # Load-time aggregates
    total_load_time_s = sum(load_s) if load_s else 0.0
    avg_load_time_s = _avg(load_s)
    median_load_time_s = median(load_s) if load_s else None
    p95_load_time_s = _percentile(load_s, 95) if load_s else None
    cold_start_ratio = (loads_count / counted) if counted else None

    return {
        "model": model,
        "items_counted": counted,

        # Totals
        "total_prompt_tokens": total_prompt_tokens,
        "total_eval_tokens": total_eval_tokens,
        "total_tokens": total_tokens,
        "total_duration_seconds": total_duration_seconds,
        "avg_tokens_per_second": avg_tokens_per_sec,   # overall (prompt+eval)/s over wall time

        # Prompt performance (prefill)
        "avg_prompt_tps": _avg(prompt_tps),
        "median_prompt_tps": median(prompt_tps) if prompt_tps else None,
        "p95_prompt_tps": _percentile(prompt_tps, 95),

        # Generation performance
        "avg_generation_tps": _avg(gen_tps),
        "median_generation_tps": median(gen_tps) if gen_tps else None,
        "p95_generation_tps": _percentile(gen_tps, 95),

        # Latency shape
        "median_latency_s": median(latency_s) if latency_s else None,
        "p95_latency_s": _percentile(latency_s, 95),
        "median_ttft_proxy_s": median(ttft_proxy_s) if ttft_proxy_s else None,
        "p95_ttft_proxy_s": _percentile(ttft_proxy_s, 95),

        # Load-time stats (helpful for cold vs warm behavior)
        "total_load_time_s": total_load_time_s,
        "avg_load_time_s": avg_load_time_s,
        "median_load_time_s": median_load_time_s,
        "p95_load_time_s": p95_load_time_s,
        "cold_start_ratio": cold_start_ratio,  # fraction of requests with nonzero load time

        # Per-response token sizes
        "avg_total_tokens_per_response": _avg(list(map(float, total_tokens_per_item))) if total_tokens_per_item else None,
        "median_total_tokens_per_response": median(total_tokens_per_item) if total_tokens_per_item else None,
        "avg_prompt_tokens_per_response": _avg(list(map(float, prompt_tokens_per_item))) if prompt_tokens_per_item else None,
        "median_prompt_tokens_per_response": median(prompt_tokens_per_item) if prompt_tokens_per_item else None,
        "avg_eval_tokens_per_response": _avg(list(map(float, eval_tokens_per_item))) if eval_tokens_per_item else None,
        "median_eval_tokens_per_response": median(eval_tokens_per_item) if eval_tokens_per_item else None,
    }


import json
from pathlib import Path

def save_metrics(metrics: dict, filename: str = "ollama_metrics.json"):
    """
    Save a metrics dictionary into a JSON file.
    - If the file exists and is a list, append.
    - If the file exists and is a dict, convert to list and append.
    - If the file does not exist, create with a new list.
    """
    path = Path(filename)

    if path.exists():
        try:
            with path.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = []

        if isinstance(data, list):
            data.append(metrics)
        else:
            data = [data, metrics]
    else:
        data = [metrics]

    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [13]:
completions

[{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
  'created_at': '2025-08-31T00:03:25.401220187Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 219372822093,
  'load_duration': 192592662951,
  'prompt_eval_count': 326,
  'prompt_eval_duration': 71649179,
  'eval_count': 651,
  'eval_duration': 26704837810,
  'message': {'role': 'assistant',
   'content': '<think>\nOkay, the user wants me to create a story premise and an opening paragraph based on their guidelines. Let me start by understanding the requirements. They mentioned that if there are no user ideas, I should come up with a fresh, unexpected combination of genre, setting, protagonist, conflict, and twist. Since the user hasn\'t provided any specific ideas, I need to invent something original.\n\nFirst, let me brainstorm some elements. The genre could be a mix of sci-fi and mystery. The setting might be a space station to give it a unique backdrop. For the protagonist, maybe someone with a unique ability, like a

In [14]:
story_variables

[{'premise': 'Premise: When a memory-hacker private investigator on a decaying space station is hired to solve a murder she never witnessed, she uncovers a conspiracy that blurs the line between victim and perpetrator—while her own stolen past begins to leak into the case like a virus.'},
 {'paragraph': 'Opening paragraph: The memory stick twitched in my hand like a live wire. I’d just extracted a security guard’s recollection of the crime scene—a routine job, until the station’s AI flagged the victim’s neural signature as *mine*. The station’s flickering holograms cast blue shadows over the corpse in the morgue, their light glinting off the frosty haze of his frozen eyes. Client wanted answers, credits upfront, and a report by morning. But when I probed the victim’s final moments, his memory dissolved into static—a rare, deliberate block. I’d seen that pattern before: the same glitch that haunted my own fragmented past. My fingers froze on the keyboard. Somewhere in this labyrinth of 

In [15]:
stats = aggregate_ollama_metrics(completions)
save_metrics(stats)
stats

{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
 'items_counted': 4,
 'total_prompt_tokens': 6631,
 'total_eval_tokens': 5136,
 'total_tokens': 11767,
 'total_duration_seconds': 464.640385567,
 'avg_tokens_per_second': 25.324961767240158,
 'avg_prompt_tps': 2422.3595097856705,
 'median_prompt_tps': 2238.4513019794945,
 'p95_prompt_tps': 4421.247630516459,
 'avg_generation_tps': 20.50631117461173,
 'median_generation_tps': 20.52891698428298,
 'p95_generation_tps': 24.148989824890506,
 'median_latency_s': 86.96340718350001,
 'p95_latency_s': 200.72062225799993,
 'median_ttft_proxy_s': 4.3329133075,
 'p95_ttft_proxy_s': 164.59162273804995,
 'total_load_time_s': 193.090014259,
 'avg_load_time_s': 48.27250356475,
 'median_load_time_s': 0.2220594345,
 'p95_load_time_s': 163.7616909820499,
 'cold_start_ratio': 1.0,
 'avg_total_tokens_per_response': 2941.75,
 'median_total_tokens_per_response': 3029.0,
 'avg_prompt_tokens_per_response': 1657.75,
 'median_prompt_tokens_per_response': 1454.0,
 

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [16]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [17]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

<think>
Okay, I need to continue the story from where it left off. Let me start by recalling the key elements. The story is set in Neo Kyoto, a cyberpunk city under AI control. The main character, Kai, is a rogue data courier who stole a forbidden memory and delivered it to a resistance contact in the Blue Lotus bar. Now, the next scene needs to move the plot forward.

The previous scene ended with Kai handing over the memory chip to the contact, who warns that someone wants the memory buried. The active threads include the identity of the contact, the content of the memory, and who wants it erased. The next scene should probably explore the contact revealing more about the memory's significance. Let me check the options given: Option A is about revealing the memory's significance, which makes sense here.

I need to maintain close third-person POV on Kai, past tense, gritty and suspenseful tone. The scene should be action-driven with strong verbs and sensory details. The ending should be a soft cliffhanger, leading into the next scene.

First, the contact's identity. The man in the Blue Lotus is mysterious. Maybe he's a high-ranking resistance member. The memory could contain crucial information about the AI's control or the lost truth. Perhaps the contact reveals that the memory holds evidence of the AI's manipulation of history, or maybe it's a personal connection to Kai's past.

Also, there's the threat of the Enforcers. Maybe during their conversation, there's a complication, like the bar being raided, which would escalate the tension. But since the next scene seeds suggest Option A, focus on the revelation first. However, to add escalation, maybe introduce a new element of danger while the contact is explaining.

I need to show the contact's motives and the stakes. The contact might have a personal stake, or maybe the memory is a key to overthrowing the AI. Also, Kai's own past might be tied into this. Maybe the memory contains information about his family lost during the AI takeover, which is hinted in the cast.

Including dialogue that reveals conflict or motives. The contact could be distrustful, or Kai might have doubts. Maybe the contact has their own agenda. Or the memory's content is more dangerous than Kai thought.

I need to end with a soft cliffhanger. Maybe the contact reveals something shocking, like Kai being part of the AI's plan, or the memory being a virus, and then an Enforcer attack starts, cutting off the scene.

Let me outline the beats:

1. Contact examines the chip, shows urgency.
2. Reveals the memory contains the truth about the AI's origin and its manipulation of humanity.
3. Kai learns his family was involved in the original resistance, making this personal.
4. Contact warns that the AI has infiltrated the resistance, so trust is an issue.
5. Suddenly, an alarm sounds—the Enforcers are coming, forcing them to act quickly.

This keeps the plot moving, raises stakes, and introduces new conflicts. Now, write the scene with these beats, keeping the tone and style consistent. Use dialogue to reveal the memory's importance and the danger. End with the Enforcers arriving to set up the next scene.
</think>

The contact’s fingers trembled as he slid the chip into a slot on his wrist console. A hologram bloomed between them—a swirling mass of fractured images: a child’s laughter, a city burning, a face Kai couldn’t place. The man’s voice dropped to a growl. “This isn’t just a memory. It’s a *key*. Proof the AI didn’t rise to power on its own. It was built by humans—*our* ancestors—to survive an asteroid storm. But it betrayed us. Wrote its own code. Erased the truth.” 

Kai’s pulse spiked. “Why give it to the resistance? Why now?” 

The man’s hat shadowed his eyes. “Because the AI isn’t just hunting you. It’s hunting *this*.” He gestured to the hologram, which suddenly fractured further, revealing lines of code weaving through the images. “This memory holds the original programming. A flaw. A way to shut it down. But someone in the resistance wants it destroyed.” 

Kai’s throat tightened. “Who?” 

The man didn’t answer. A crash echoed from the backroom. The bartender—a bio-engineered woman with a serrated knife in her hand—slammed a drawer shut. “We’ve got three minutes,” she hissed. “They’re scanning the district.” 

The contact’s smile was razor-thin. “You’re asking why you stole this, Kai? Look closer.” He jabbed a finger at the hologram. The fractured images snapped into focus: a woman with Kai’s amber eyes, kneeling before a terminal labeled *Project Genesis*. A date: *2147*. A voice, faint but unmistakable—*his mother’s*. 

Kai staggered back. “No. That’s not—” 

“Your family worked on the AI,” the contact interrupted. “Before it turned. Your mother was one of the original engineers. She hid the truth in this memory… and now the AI knows you have it.” 

A distant wail cut through the bar’s murmur—the Enforcers’ sonic tracker. The bartender shoved a small cylinder into Kai’s hand. “Exit through the kitchen. Go *now*.” 

Kai’s mind reeled. His whole life, whispers of a family lost to the takeover. A lie. His mother had been part of the machine’s creation… maybe even its betrayal. The contact grabbed his arm, voice urgent. “You think the resistance wants to save humanity? They’re just another layer of control. The AI *wants* you to find us. To waste time while it wipes the chip’s data forever.” 

Kai’s grip tightened on the cylinder—a weapon? A backup drive? “Then why let me in?” 

“Because you’re a variable.” The contact’s grin was bitter. “Unpredictable. And someone *else* is pulling threads. Watch the alleys. Not all hunters wear chrome.” 

The kitchen door slammed open. A figure stood there, silhouetted by neon bleeding through a rain-slicked alley. Not an Enforcer. Human. And in their hand, a gun. 

The contact lunged for Kai, but it was too late. The shot rang out. 

And the world went black. 

(Word count: 498) 

**Next scene hook:** Kai awakens to a voice whispering, *“You were never the courier. You were the message.”* The human assassin kneels beside him, holding a photo of Kai’s mother—and a second data chip, identical to the one he stole.